In [25]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load an official model

In [48]:
validation_results = model.val(data='coco8.yaml',
                               imgsz=640,
                               batch=16,
                               conf=0.25,
                               iou=0.6,
                               device='0')

Ultralytics YOLOv8.1.23 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20154MiB)


val: Scanning /root/data-flywheel/notebooks/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.73it/s]


                   all          4         17      0.833       0.65      0.739       0.51
                person          4         10          1        0.4        0.7       0.42
                   dog          4          1          1          1      0.995      0.597
                 horse          4          2          1          1      0.995      0.672
              elephant          4          2          1        0.5       0.75      0.375
              umbrella          4          1          1          1      0.995      0.995
          potted plant          4          1          0          0          0          0
Speed: 0.2ms preprocess, 8.9ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val3


In [56]:
# Run validation
model.fit()
metrics = model.val(data='coco8.yaml')

# Access the losses
box_loss = metrics.box.loss
cls_loss = metrics.cls.loss
dfl_loss = metrics.dfl.loss

# Print the losses
print(f"Validation Box Loss: {box_loss}")
print(f"Validation Class Loss: {cls_loss}")
print(f"Validation Distribution Focal Loss: {dfl_loss}")

AttributeError: 'YOLO' object has no attribute 'fit'

In [33]:
import cv2

img_path = '/workspace/datasets/coco_minitrain_25k/images/train2017/000000231538.jpg'
image = cv2.imread(img_path)
preds = model(image, verbose=False)[0]

In [45]:
bboxes = preds.boxes.xywh
labels = preds.boxes.cls

In [ ]:
targets = {
    'bboxes': bboxes,  # Your ground truth boxes
    'labels': labels,  # Your ground truth labels
    'batch_idx': torch.zeros(labels.size(0)).long()  # Assuming all targets belong to the first (and only) image
}

In [46]:
import torch
from torch import nn

In [47]:
from ultralytics.utils.loss import v8DetectionLoss

loss_function = v8DetectionLoss(model.model)  # Initialized with your model

In [ ]:
loss_function()

In [58]:
from ultralytics.models.yolo.detect import DetectionValidator

args = dict(model='yolov8n.pt', data='coco8.yaml')
validator = DetectionValidator(args=args)
validator()

Ultralytics YOLOv8.1.23 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20154MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /root/data-flywheel/notebooks/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.27it/s]


                   all          4         17      0.709      0.833      0.809      0.612
                person          4         10      0.783        0.5      0.541      0.278
                   dog          4          1      0.452          1      0.497      0.348
                 horse          4          2      0.819          1      0.995      0.747
              elephant          4          2      0.896        0.5      0.828      0.407
              umbrella          4          1      0.636          1      0.995      0.995
          potted plant          4          1      0.668          1      0.995      0.895
Speed: 0.1ms preprocess, 62.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


{'metrics/precision(B)': 0.7088189260125325,
 'metrics/recall(B)': 0.8333333333333334,
 'metrics/mAP50(B)': 0.8085673514895743,
 'metrics/mAP50-95(B)': 0.6117262163876994,
 'fitness': 0.6314103298978869}